In [15]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

mnist = fetch_openml('mnist_784', version=1)


X = mnist.data
y = mnist.target.astype(np.uint8)  

In [16]:
subset_size = 1000  
indices = np.random.choice(len(X), subset_size, replace=False)
X_subset = X.iloc[indices]
y_subset = y.iloc[indices]

In [17]:
# Scale the features to zero mean and unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.astype(np.float64))

In [18]:
# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=1/7, random_state=42
)

In [19]:
# Step 4: Initialize the SVM classifier
svm_clf = LinearSVC(max_iter=20000, random_state=42)

In [ ]:
# Step 5: Train the classifier
svm_clf.fit(X_train, y_train)



c:\Users\Jean\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [6]:
# Step 6: Make predictions on the test set
y_pred = svm_clf.predict(X_test)


AttributeError: 'LinearSVC' object has no attribute 'coef_'

In [ ]:

# Step 7: Evaluate the classifier
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Overall Accuracy:", accuracy_score(y_test, y_pred))

### Visualisation

In [ ]:
import matplotlib.pyplot as plt

# Find indices of misclassified samples
misclassified_indices = np.where(y_test != y_pred)[0]

# Plot some misclassified samples
plt.figure(figsize=(10, 4))
for i, index in enumerate(misclassified_indices[:5]):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_test[index].reshape(28, 28), cmap='gray')
    plt.title(f"True: {y_test[index]}\nPred: {y_pred[index]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


# Grid Search

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import warnings
warnings.filterwarnings('ignore') 

In [14]:
mnist = fetch_openml('mnist_784', version=1)

X = mnist.data
y = mnist.target.astype(np.uint8) 

subset_size = 1000 
indices = np.random.choice(len(X), subset_size, replace=False)
X = X.iloc[indices]
y = y.iloc[indices]

# Scale the features to zero mean and unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.astype(np.float64))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [ ]:
param_grid_linear = {
    'C': [0.1, 1, 10]
}

param_grid_poly = {
    'C': [0.1, 1, 10],
    'degree': [2, 3],
    'coef0': [0, 1],
    'kernel': ['poly']
}

param_grid_rbf = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['rbf']
}


### Linear kernel

In [ ]:
print("Starting Grid Search for Linear Kernel...")
grid_search_linear = GridSearchCV(
    estimator=SVC(kernel='linear'),
    param_grid=param_grid_linear,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

start_time = time.time()
grid_search_linear.fit(X_train, y_train)
end_time = time.time()

print(f"Linear Kernel Grid Search took {end_time - start_time:.2f} seconds.")
print("Best Parameters for Linear Kernel:", grid_search_linear.best_params_)
print(f"Best Cross-Validation Accuracy: {grid_search_linear.best_score_:.4f}\n")


### Polynomial Kernel

In [ ]:
print("Starting Grid Search for Polynomial Kernel...")
grid_search_poly = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid_poly,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

start_time = time.time()
grid_search_poly.fit(X_train, y_train)
end_time = time.time()

print(f"Polynomial Kernel Grid Search took {end_time - start_time:.2f} seconds.")
print("Best Parameters for Polynomial Kernel:", grid_search_poly.best_params_)
print(f"Best Cross-Validation Accuracy: {grid_search_poly.best_score_:.4f}\n")


### RBF Kernel

In [ ]:
print("Starting Grid Search for RBF Kernel...")
grid_search_rbf = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid_rbf,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

start_time = time.time()
grid_search_rbf.fit(X_train, y_train)
end_time = time.time()

print(f"RBF Kernel Grid Search took {end_time - start_time:.2f} seconds.")
print("Best Parameters for RBF Kernel:", grid_search_rbf.best_params_)
print(f"Best Cross-Validation Accuracy: {grid_search_rbf.best_score_:.4f}\n")


### Evaluate best model on test set

In [ ]:
# Initialize a dictionary to store evaluation metrics
kernel_results = {}

# Linear Kernel Evaluation
best_linear_clf = grid_search_linear.best_estimator_
y_pred_linear = best_linear_clf.predict(X_test)
accuracy_linear = accuracy_score(y_test, y_pred_linear)
report_linear = classification_report(y_test, y_pred_linear, output_dict=True)
kernel_results['linear'] = {
    'accuracy': accuracy_linear,
    'report': report_linear,
    'confusion_matrix': confusion_matrix(y_test, y_pred_linear)
}

# Polynomial Kernel Evaluation
best_poly_clf = grid_search_poly.best_estimator_
y_pred_poly = best_poly_clf.predict(X_test)
accuracy_poly = accuracy_score(y_test, y_pred_poly)
report_poly = classification_report(y_test, y_pred_poly, output_dict=True)
kernel_results['poly'] = {
    'accuracy': accuracy_poly,
    'report': report_poly,
    'confusion_matrix': confusion_matrix(y_test, y_pred_poly)
}

# RBF Kernel Evaluation
best_rbf_clf = grid_search_rbf.best_estimator_
y_pred_rbf = best_rbf_clf.predict(X_test)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
report_rbf = classification_report(y_test, y_pred_rbf, output_dict=True)
kernel_results['rbf'] = {
    'accuracy': accuracy_rbf,
    'report': report_rbf,
    'confusion_matrix': confusion_matrix(y_test, y_pred_rbf)
}


### compare perfomance of different kernel

In [ ]:
print("Test Set Accuracy for Different Kernels:")
for kernel in kernel_results:
    print(f"{kernel.capitalize()} Kernel Accuracy: {kernel_results[kernel]['accuracy']:.4f}")


In [ ]:
for kernel in kernel_results:
    print(f"\nClassification Report for {kernel.capitalize()} Kernel:")
    print(classification_report(y_test, globals()[f'y_pred_{kernel}']))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for kernel in kernel_results:
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        kernel_results[kernel]['confusion_matrix'],
        annot=True, fmt='d', cmap='Blues',
        xticklabels=np.unique(y), yticklabels=np.unique(y)
    )
    plt.title(f"Confusion Matrix for {kernel.capitalize()} Kernel")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


### best model


In [ ]:
# Find the kernel with the highest accuracy
best_kernel = max(kernel_results, key=lambda k: kernel_results[k]['accuracy'])
print(f"\nThe best kernel is: {best_kernel.capitalize()} Kernel with accuracy {kernel_results[best_kernel]['accuracy']:.4f}")
